In [ ]:
#Require 1.1 Data Extraction and Transformation with Python and PySpark 
import json

# Input this Function to read JSON file:
def read_json(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

file = ("C:/Users/Learner_9ZH3Z104/Downloads/Mapping Document.xlsx+")

# Reading JSON files
branches = read_json('C:/Users/Learner_9ZH3Z104/Development/Capstone/LatestDataSet/cdw_sapp_branch.json')
credit_cards = read_json('C:/Users/Learner_9ZH3Z104/Development/Capstone/LatestDataSet/cdw_sapp_credit.json')
customers = read_json('C:/Users/Learner_9ZH3Z104/Development/Capstone/LatestDataSet/cdw_sapp_credit.json')

#1.2 Data loading into Database 
import mysql.connector
from mysql.connector import Error
import json

# Paths to your JSON files
branch_json_file_path = 'C:/Users/Learner_9ZH3Z104/Development/Capstone/LatestDataSet/cdw_sapp_branch.json'
credit_json_file_path = 'C:/Users/Learner_9ZH3Z104/Development/Capstone/LatestDataSet/cdw_sapp_credit.json'
customer_json_file_path = 'C:/Users/Learner_9ZH3Z104/Development/Capstone/LatestDataSet/cdw_sapp_customer.json'

try:
    # Load data from JSON files
    with open(branch_json_file_path, 'r') as branch_file:
        branches_data = json.load(branch_file)
    
    with open(credit_json_file_path, 'r') as credit_file:
        credit_cards_data = json.load(credit_file)
    
    with open(customer_json_file_path, 'r') as customer_file:
        customers_data = json.load(customer_file)

    connection = mysql.connector.connect(
        host='localhost',
        user='root',
        password='Password',  # Replace with your MySQL password
        port='3306'  # Port should be an integer
    )

    if connection.is_connected():
        cursor = connection.cursor()

        # Drop the existing 'creditcard_capstone' database if it exists
        cursor.execute("DROP DATABASE IF EXISTS creditcard_capstone")
        print("Existing database 'creditcard_capstone' dropped successfully.")

        # Create the new 'creditcard_capstone' database
        cursor.execute("CREATE DATABASE creditcard_capstone")
        print("New database 'creditcard_capstone' created successfully.")

        # Switch to the new database
        cursor.execute("USE creditcard_capstone")

        # Create tables for the Credit Card System
        create_branch_table_query = """
        CREATE TABLE IF NOT EXISTS CDW_SAPP_BRANCH (
            branch_code INT PRIMARY KEY,
            branch_name VARCHAR(255),
            branch_street VARCHAR(255),
            branch_city VARCHAR(255),
            branch_state VARCHAR(255),
            branch_zip VARCHAR(20),
            branch_phone VARCHAR(20),
            last_updated DATE
        )
        """
        cursor.execute(create_branch_table_query)

        create_credit_card_table_query = """
        CREATE TABLE IF NOT EXISTS CDW_SAPP_CREDIT_CARD (
            transaction_id INT PRIMARY KEY,
            day INT,
            month INT,
            year INT,
            credit_card_no VARCHAR(255),
            cust_ssn INT,
            branch_code INT,
            transaction_type VARCHAR(255),
            transaction_value DECIMAL(10, 2)
        )
        """
        cursor.execute(create_credit_card_table_query)

        create_customer_table_query = """
        CREATE TABLE IF NOT EXISTS CDW_SAPP_CUSTOMER (
            ssn INT PRIMARY KEY,
            first_name VARCHAR(255),
            middle_name VARCHAR(255),
            last_name VARCHAR(255),
            credit_card_no VARCHAR(255),
            apt_no VARCHAR(10),
            street_name VARCHAR(255),
            cust_city VARCHAR(255),
            cust_state VARCHAR(255),
            cust_country VARCHAR(255),
            cust_zip VARCHAR(20),
            cust_phone VARCHAR(20),
            cust_email VARCHAR(255),
            last_updated DATETIME
        )
        """
        cursor.execute(create_customer_table_query)

        print("Tables have been created successfully.")

        # Insert data into CDW_SAPP_BRANCH table from JSON file
        for row in branches_data:
            cursor.execute("""
                INSERT INTO CDW_SAPP_BRANCH (
                    branch_code, branch_name, branch_street, branch_city,
                    branch_state, branch_zip, branch_phone, last_updated
                ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
            """, (
                row['BRANCH_CODE'], row['BRANCH_NAME'], row['BRANCH_STREET'],
                row['BRANCH_CITY'], row['BRANCH_STATE'], row['BRANCH_ZIP'],
                row['BRANCH_PHONE'], row['LAST_UPDATED']
            ))
        
        # Insert data into CDW_SAPP_CUSTOMER table from JSON file
        for row in customers_data:
            cursor.execute("""
                INSERT INTO CDW_SAPP_CUSTOMER (
                    ssn, first_name, middle_name, last_name, credit_card_no, apt_no, 
                    street_name, cust_city, cust_state, cust_country, cust_zip, 
                    cust_phone, cust_email, last_updated
                ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """, (
                row['SSN'], row['FIRST_NAME'], row['MIDDLE_NAME'], row['LAST_NAME'], 
                row['CREDIT_CARD_NO'], row['APT_NO'], row['STREET_NAME'], row['CUST_CITY'], 
                row['CUST_STATE'], row['CUST_COUNTRY'], row['CUST_ZIP'], row['CUST_PHONE'], 
                row['CUST_EMAIL'], row['LAST_UPDATED']
            ))
        
        # Insert data into CDW_SAPP_CREDIT_CARD table from JSON file
        for row in credit_cards_data:
            cursor.execute("""
                INSERT INTO CDW_SAPP_CREDIT_CARD (
                    transaction_id, day, month, year, credit_card_no, 
                    cust_ssn, branch_code, transaction_type, transaction_value
                ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
            """, (
                row['TRANSACTION_ID'], row['DAY'], row['MONTH'], row['YEAR'], 
                row['CREDIT_CARD_NO'], row['CUST_SSN'], row['BRANCH_CODE'], 
                row['TRANSACTION_TYPE'], row['TRANSACTION_VALUE']
            ))

        connection.commit()
        print("Data has been inserted successfully.")

    else:
        print("Connection failed.")

except Error as e:
    print(f"Error: {e}")
except json.JSONDecodeError as je:
    print(f"Error reading JSON file: {je}")
except FileNotFoundError as fe:
    print(f"JSON file not found: {fe}")

finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection is closed.")

